Coding Exercise

- Data source: free API access from Coinapi.io
- API documentation: https://docs.coinapi.io/

### Set up

In [225]:
import requests
import numpy as np
import pandas as pd
import datetime as dt
import pytz
import time
import gdax

pd.options.mode.chained_assignment = None

In [196]:
'''
# limit range from 100 to 10000
limit = 1000

# Create trades table storing all trades
trades = pd.DataFrame()

# free api key (limit: 100 request/day) can be requested at: https://www.coinapi.io/pricing?apikey
api_key = '8E41560E-D4D2-49E2-9BDE-AD7ABC04005C' 

url = 'https://rest.coinapi.io/v1/trades/COINBASE_SPOT_BTC_USD/latest?limit=' + str(limit)
headers = {'X-CoinAPI-Key' : api_key}
'''

"\n# limit range from 100 to 10000\nlimit = 1000\n\n# Create trades table storing all trades\ntrades = pd.DataFrame()\n\n# free api key (limit: 100 request/day) can be requested at: https://www.coinapi.io/pricing?apikey\napi_key = '8E41560E-D4D2-49E2-9BDE-AD7ABC04005C' \n\nurl = 'https://rest.coinapi.io/v1/trades/COINBASE_SPOT_BTC_USD/latest?limit=' + str(limit)\nheaders = {'X-CoinAPI-Key' : api_key}\n"

In [197]:
'''
url = 'http://marketdata.wavesplatform.com/api/trades/BTC/USD/500'
response = requests.get(url)
trades_new = pd.DataFrame(response.json())
trades_new['datetime'] = trades_new['timestamp'].apply(lambda t: dt.datetime.utcfromtimestamp(t/1000))
trades_new.head()
'''

"\nurl = 'http://marketdata.wavesplatform.com/api/trades/BTC/USD/500'\nresponse = requests.get(url)\ntrades_new = pd.DataFrame(response.json())\ntrades_new['datetime'] = trades_new['timestamp'].apply(lambda t: dt.datetime.utcfromtimestamp(t/1000))\ntrades_new.head()\n"

In [198]:
# Stole code from https://github.com/danpaquin/gdax-python

#
# GDAX/PublicClient.py
# Daniel Paquin
#
# For public requests to the GDAX exchange

import requests


class PublicClient(object):
    """GDAX public client API.
    All requests default to the `product_id` specified at object
    creation if not otherwise specified.
    Attributes:
        url (Optional[str]): API URL. Defaults to GDAX API.
    """

    def __init__(self, api_url='https://api.gdax.com', timeout=30):
        """Create GDAX API public client.
        Args:
            api_url (Optional[str]): API URL. Defaults to GDAX API.
        """
        self.url = api_url.rstrip('/')
        self.timeout = timeout

    def _get(self, path, params=None):
        """Perform get request"""

        r = requests.get(self.url + path, params=params, timeout=self.timeout)
        # r.raise_for_status()
        return r.json()

    def get_products(self):
        """Get a list of available currency pairs for trading.
        Returns:
            list: Info about all currency pairs. Example::
                [
                    {
                        "id": "BTC-USD",
                        "display_name": "BTC/USD",
                        "base_currency": "BTC",
                        "quote_currency": "USD",
                        "base_min_size": "0.01",
                        "base_max_size": "10000.00",
                        "quote_increment": "0.01"
                    }
                ]
        """
        return self._get('/products')

    def get_product_order_book(self, product_id, level=1):
        """Get a list of open orders for a product.
        The amount of detail shown can be customized with the `level`
        parameter:
        * 1: Only the best bid and ask
        * 2: Top 50 bids and asks (aggregated)
        * 3: Full order book (non aggregated)
        Level 1 and Level 2 are recommended for polling. For the most
        up-to-date data, consider using the websocket stream.
        **Caution**: Level 3 is only recommended for users wishing to
        maintain a full real-time order book using the websocket
        stream. Abuse of Level 3 via polling will cause your access to
        be limited or blocked.
        Args:
            product_id (str): Product
            level (Optional[int]): Order book level (1, 2, or 3).
                Default is 1.
        Returns:
            dict: Order book. Example for level 1::
                {
                    "sequence": "3",
                    "bids": [
                        [ price, size, num-orders ],
                    ],
                    "asks": [
                        [ price, size, num-orders ],
                    ]
                }
        """

        # Supported levels are 1, 2 or 3
        level = level if level in range(1, 4) else 1
        return self._get('/products/{}/book'.format(str(product_id)), params={'level': level})

    def get_product_ticker(self, product_id):
        """Snapshot about the last trade (tick), best bid/ask and 24h volume.
        **Caution**: Polling is discouraged in favor of connecting via
        the websocket stream and listening for match messages.
        Args:
            product_id (str): Product
        Returns:
            dict: Ticker info. Example::
                {
                  "trade_id": 4729088,
                  "price": "333.99",
                  "size": "0.193",
                  "bid": "333.98",
                  "ask": "333.99",
                  "volume": "5957.11914015",
                  "time": "2015-11-14T20:46:03.511254Z"
                }
        """
        return self._get('/products/{}/ticker'.format(str(product_id)))

    def get_product_trades(self, product_id, before='', after='', limit=100, result=[]):
        """List the latest trades for a product.
        Args:
             product_id (str): Product
             before (Optional[str]): start time in ISO 8601
             after (Optional[str]): end time in ISO 8601
             limit (Optional[int]): the desired number of trades (can be more than 100,
                          automatically paginated)
             results (Optional[list]): list of results that is used for the pagination
        Returns:
             list: Latest trades. Example::
                 [{
                     "time": "2014-11-07T22:19:28.578544Z",
                     "trade_id": 74,
                     "price": "10.00000000",
                     "size": "0.01000000",
                     "side": "buy"
                 }, {
                     "time": "2014-11-07T01:08:43.642366Z",
                     "trade_id": 73,
                     "price": "100.00000000",
                     "size": "0.01000000",
                     "side": "sell"
         }]
        """
        url = self.url + '/products/{}/trades'.format(str(product_id))
        params = {}

        if before:
            params['before'] = str(before)
        if after:
            params['after'] = str(after)
        if limit and limit < 100:
            # the default limit is 100
            # we only add it if the limit is less than 100
            params['limit'] = limit

        r = requests.get(url, params=params)
        # r.raise_for_status()

        result.extend(r.json())

        if 'cb-after' in r.headers and limit is not len(result):
            # update limit
            limit = limit - len(result)
            if limit <= 0:
                return result

            # TODO: need a way to ensure that we don't get rate-limited/blocked
            # time.sleep(0.4)
            return self.get_product_trades(product_id=product_id, after=r.headers['cb-after'], limit=limit, result=result)

        return result

    def get_product_historic_rates(self, product_id, start=None, end=None,
                                   granularity=None):
        """Historic rates for a product.
        Rates are returned in grouped buckets based on requested
        `granularity`. If start, end, and granularity aren't provided,
        the exchange will assume some (currently unknown) default values.
        Historical rate data may be incomplete. No data is published for
        intervals where there are no ticks.
        **Caution**: Historical rates should not be polled frequently.
        If you need real-time information, use the trade and book
        endpoints along with the websocket feed.
        The maximum number of data points for a single request is 200
        candles. If your selection of start/end time and granularity
        will result in more than 200 data points, your request will be
        rejected. If you wish to retrieve fine granularity data over a
        larger time range, you will need to make multiple requests with
        new start/end ranges.
        Args:
            product_id (str): Product
            start (Optional[str]): Start time in ISO 8601
            end (Optional[str]): End time in ISO 8601
            granularity (Optional[str]): Desired time slice in seconds
        Returns:
            list: Historic candle data. Example::
                [
                    [ time, low, high, open, close, volume ],
                    [ 1415398768, 0.32, 4.2, 0.35, 4.2, 12.3 ],
                    ...
                ]
        """
        params = {}
        if start is not None:
            params['start'] = start
        if end is not None:
            params['end'] = end
        if granularity is not None:
            acceptedGrans = [60, 300, 900, 3600, 21600, 86400]
            if granularity not in acceptedGrans:
                newGranularity = min(acceptedGrans, key=lambda x:abs(x-granularity))
                print(granularity,' is not a valid granularity level, using',newGranularity,' instead.')
                granularity = newGranularity
            params['granularity'] = granularity

        return self._get('/products/{}/candles'.format(str(product_id)), params=params)

    def get_product_24hr_stats(self, product_id):
        """Get 24 hr stats for the product.
        Args:
            product_id (str): Product
        Returns:
            dict: 24 hour stats. Volume is in base currency units.
                Open, high, low are in quote currency units. Example::
                    {
                        "open": "34.19000000",
                        "high": "95.70000000",
                        "low": "7.06000000",
                        "volume": "2.41000000"
                    }
        """
        return self._get('/products/{}/stats'.format(str(product_id)))

    def get_currencies(self):
        """List known currencies.
        Returns:
            list: List of currencies. Example::
                [{
                    "id": "BTC",
                    "name": "Bitcoin",
                    "min_size": "0.00000001"
                }, {
                    "id": "USD",
                    "name": "United States Dollar",
                    "min_size": "0.01000000"
                }]
        """
        return self._get('/currencies')

    def get_time(self):
        """Get the API server time.
        Returns:
            dict: Server time in ISO and epoch format (decimal seconds
                since Unix epoch). Example::
                    {
                        "iso": "2015-01-07T23:47:25.201Z",
                        "epoch": 1420674445.201
                    }
        """
        return self._get('/time')

In [ ]:
def __init__(self, api_url='https://api.gdax.com', timeout=30):
        """Create GDAX API public client.
        Args:
            api_url (Optional[str]): API URL. Defaults to GDAX API.
        """
        self.url = 'https://api.gdax.com'.rstrip('/')
        self.timeout = timeout


In [441]:
'https://api.gdax.com'.rstrip('/')

'https://api.gdax.com'

In [538]:
def get_product_trades(product_id, before='', after='', limit=100, result=[]):
    """List the latest trades for a product.
    Args:
         product_id (str): Product
         before (Optional[str]): start time in ISO 8601
         after (Optional[str]): end time in ISO 8601
         limit (Optional[int]): the desired number of trades (can be more than 100,
                      automatically paginated)
         results (Optional[list]): list of results that is used for the pagination
    Returns:
         list: Latest trades. Example::
             [{
                 "time": "2014-11-07T22:19:28.578544Z",
                 "trade_id": 74,
                 "price": "10.00000000",
                 "size": "0.01000000",
                 "side": "buy"
             }, {
                 "time": "2014-11-07T01:08:43.642366Z",
                 "trade_id": 73,
                 "price": "100.00000000",
                 "size": "0.01000000",
                 "side": "sell"
     }]
    """
    url = 'https://api.gdax.com/products/{}/trades'.format(str(product_id))
    params = {}

    if before:
        params['before'] = str(before)
    if after:
        params['after'] = str(after)
    if limit and limit < 100:
        # the default limit is 100
        # we only add it if the limit is less than 100
        params['limit'] = limit

    r = requests.get(url, params=params)
    
    result.extend(r.json())

    if 'cb-after' in r.headers and limit is not len(result):
        # update limit
        limit = limit - len(result)
        if limit <= 0:
            return result

        # TODO: need a way to ensure that we don't get rate-limited/blocked
        time.sleep(0.4)
        return get_product_trades(product_id=product_id, after=r.headers['cb-after'], limit=limit, result=result)

    return result

In [564]:
trades_raw = get_product_trades(product_id='BTC-USD', limit=100)

In [565]:
trades_new = pd.DataFrame(trades_raw)

In [566]:
trades_new

,price,side,size,time,trade_id
0,7291.87000000,sell,0.03361339,2018-07-19T02:10:56.812Z,46737960
1,7291.87000000,sell,6.19671337,2018-07-19T02:10:51.42Z,46737959
2,7291.87000000,sell,0.01694383,2018-07-19T02:10:51.259Z,46737958
3,7291.87000000,sell,0.53781426,2018-07-19T02:10:49.129Z,46737957
4,7291.86000000,buy,0.17402623,2018-07-19T02:10:44.921Z,46737956
5,7291.87000000,sell,0.03542963,2018-07-19T02:10:44.305Z,46737955
6,7291.86000000,buy,0.05776262,2018-07-19T02:10:44.097Z,46737954
7,7291.87000000,sell,0.00122948,2018-07-19T02:10:43.901Z,46737953
8,7291.87000000,sell,0.00846168,2018-07-19T02:10:41.416Z,46737952
9,7291.86000000,buy,0.00330000,2018-07-19T02:10:32.9Z,46737951


In [567]:
len(trades_new['trade_id'].unique())

218

In [800]:
r = requests.get('https://api.gdax.com/products/BTC-USD/trades')
a = pd.DataFrame(r.json()).set_index('trade_id')
time.sleep(10)
r = requests.get('https://api.gdax.com/products/BTC-USD/trades')
b = pd.DataFrame(r.json()).set_index('trade_id')

ConnectionError: HTTPSConnectionPool(host='api.gdax.com', port=443): Max retries exceeded with url: /products/BTC-USD/trades (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x11e8d2d68>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [632]:
merged = a.reset_index().merge(b.reset_index(), indicator=True, how='outer').set_index('trade_id')
merged

,price,side,size,time,_merge
trade_id,,,,,
46740987,7327.31000000,sell,0.00570384,2018-07-19T03:04:35.121Z,both
46740986,7327.31000000,sell,0.00448265,2018-07-19T03:04:34.905Z,both
46740985,7327.30000000,buy,0.10000000,2018-07-19T03:04:31.675Z,both
46740984,7327.31000000,sell,0.01631996,2018-07-19T03:04:23.535Z,both
46740983,7327.30000000,buy,0.03770655,2018-07-19T03:04:21.716Z,both
46740982,7328.46000000,buy,0.01250000,2018-07-19T03:04:21.716Z,both
46740981,7328.56000000,buy,0.02731009,2018-07-19T03:04:21.716Z,both
46740980,7328.56000000,buy,0.00098336,2018-07-19T03:04:21.716Z,both
46740979,7328.56000000,buy,0.00032664,2018-07-19T03:04:17.533Z,both


In [685]:
merged[merged['_merge'] == 'right_only'].drop('_merge', axis=1)

,price,side,size,time
trade_id,,,,
46740993,7327.31000000,sell,0.03283565,2018-07-19T03:04:46.575Z
46740992,7327.30000000,buy,0.02715665,2018-07-19T03:04:46.101Z
46740991,7327.31000000,sell,0.00353207,2018-07-19T03:04:44.796Z
46740990,7327.31000000,sell,0.02675332,2018-07-19T03:04:43.997Z
46740989,7327.30000000,buy,0.00140000,2018-07-19T03:04:41.361Z
46740988,7327.31000000,sell,0.05500000,2018-07-19T03:04:38.736Z


In [696]:
def parties_assign(n_parties, table1, table2=None):
    """
    Assign buyers and sellers to each trade

    args:
        n_parties: total number of counter parties
        table: the table of trades
    """
    # print('parties_assign received table:')

    parties_list = []
    
    if table2 is not None:
        merged = table1.reset_index().merge(table2.reset_index(),
                                            indicator=True,
                                            how='outer').set_index('trade_id')
        new_rows = merged[merged['_merge'] == 'right_only'].drop('_merge',
                                                                 axis=1)
        
        if len(merged) > 0:
            
            for i in np.arange(len(new_rows)):
                parties_list.append(np.random.choice(10, 2, replace=False))
                
            parties_list = np.array(parties_list)
            
            new_rows['Buyer'] = parties_list[:,0]
            new_rows['Seller'] = parties_list[:,1]

            table1 = pd.concat([table1, new_rows])

    else:
        for i in np.arange(len(table1)):
            parties_list.append(np.random.choice(n_parties, 2, replace=False))
        
        parties_list = np.array(parties_list)

        table1['Buyer'] = parties_list[:, 0]
        table1['Seller'] = parties_list[:, 1]
                
    return table1

In [699]:
trade_data = parties_assign(10, a, b)
trade_data.head()

,price,side,size,time,Buyer,Seller
trade_id,,,,,,
46740987,7327.31000000,sell,0.00570384,2018-07-19T03:04:35.121Z,0,4
46740986,7327.31000000,sell,0.00448265,2018-07-19T03:04:34.905Z,9,8
46740985,7327.30000000,buy,0.10000000,2018-07-19T03:04:31.675Z,1,0
46740984,7327.31000000,sell,0.01631996,2018-07-19T03:04:23.535Z,3,7
46740983,7327.30000000,buy,0.03770655,2018-07-19T03:04:21.716Z,9,6


In [714]:
def iso_converter(s):
    """
    Convert a string to datetime object, and deal with exceptions possibly
    causing bugs.

    args:
        s: a string in iso format

    return: a datetime object
    """
    if len(s) == 20:
        t = dt.datetime.strptime(s, "%Y-%m-%dT%H:%M:%SZ")
    else:
        t = dt.datetime.strptime(s, "%Y-%m-%dT%H:%M:%S.%fZ")

    return t

In [813]:
trade_data['time'][46740987]
dt.timestamp('2018-07-19 03:04:35.121000')

AttributeError: module 'datetime' has no attribute 'timestamp'

In [715]:
trade_data['time'] = trade_data['time'].apply(iso_converter)

In [735]:
# Take the data of lastest hour
last_hr_data = trade_data[trade_data['time'] > dt.datetime.utcnow() - dt.timedelta(hours=2)]

# Calculate trade worth
last_hr_data['price'] = last_hr_data['price'].astype(float)
last_hr_data['size'] = last_hr_data['size'].astype(float)
last_hr_data['trade_worth'] = last_hr_data['price'] * last_hr_data['size']
last_hr_data.head()

,price,side,size,time,Buyer,Seller,vwap_value,trade_worth
trade_id,,,,,,,,
46740987,7327.31,sell,0.005704,2018-07-19 03:04:35.121,0,4,41.793804,41.793804
46740986,7327.31,sell,0.004483,2018-07-19 03:04:34.905,9,8,32.845766,32.845766
46740985,7327.30,buy,0.100000,2018-07-19 03:04:31.675,1,0,732.730000,732.730000
46740984,7327.31,sell,0.016320,2018-07-19 03:04:23.535,3,7,119.581406,119.581406
46740983,7327.30,buy,0.037707,2018-07-19 03:04:21.716,9,6,276.287204,276.287204


In [745]:
total_worth = last_hr_data.groupby('Buyer')['vwap_value'].sum() - trade_data.groupby('Seller')['vwap_value'].sum()
total_worth

Buyer
0    -3264.898311
1    21645.851495
2    53979.931280
3   -13902.415925
4      839.541497
5     -547.928920
6    -7515.188604
7     7357.365566
8   -50837.842481
9    -7754.415597
Name: vwap_value, dtype: float64

In [746]:
total_quantity = last_hr_data.groupby('Buyer')['size'].sum() - last_hr_data.groupby('Seller')['size'].sum()
total_quantity

Buyer
0   -0.445524
1    2.953646
2    7.365616
3   -1.897010
4    0.114533
5   -0.074765
6   -1.025463
7    1.003928
8   -6.936949
9   -1.058011
Name: size, dtype: float64

In [791]:
settlements = pd.DataFrame(total_worth/total_quantity, columns=['vwap_value'])
settlements['quantity'] = total_quantity
settlements.index.names = ['Party']

r = requests.get('https://api.gdax.com/products/{}/ticker'.format(str('BTC-USD')))
settlements['current_price'] = float(r.json()['price'])

settlements['settlement_obligation'] = (settlements['vwap_value'] - settlements['current_price']) * settlements['quantity']

settlements

,vwap_value,quantity,current_price,settlement_obligation
Party,,,,
0,7328.222926,-0.445524,7331.97,1.669411
1,7328.520342,2.953646,7331.97,-10.189068
2,7328.638192,7.365616,7331.97,-24.540817
3,7328.594794,-1.897010,7331.97,6.402799
4,7330.133294,0.114533,7331.97,-0.210363
5,7328.660573,-0.074765,7331.97,0.247430
6,7328.582877,-1.025463,7331.97,3.473368
7,7328.582485,1.003928,7331.97,-3.400819
8,7328.559206,-6.936949,7331.97,23.660505


In [787]:
old_settlements = settlements
old_settlements

,vwap_value,quantity,current_price,settlement_obligation
Party,,,,
0,7328.222926,-0.445524,7327.59,-0.281983
1,7328.520342,2.953646,7327.59,2.747899
2,7328.638192,7.365616,7327.59,7.720579
3,7328.594794,-1.897010,7327.59,-1.906104
4,7330.133294,0.114533,7327.59,0.291291
5,7328.660573,-0.074765,7327.59,-0.080042
6,7328.582877,-1.025463,7327.59,-1.018159
7,7328.582485,1.003928,7327.59,0.996383
8,7328.559206,-6.936949,7327.59,-6.723332


In [795]:
new_obligations = settlements['settlement_obligation'] - old_settlements['settlement_obligation']
new_obligations

Party
0     1.951395
1   -12.936968
2   -32.261396
3     8.308903
4    -0.501654
5     0.327472
6     4.491527
7    -4.397202
8    30.383837
9     4.634087
Name: settlement_obligation, dtype: float64

In [797]:
balances = pd.DataFrame({'Party': range(10),
                                 'Initial Balance': 100000,
                                 'Current Balance': 100000}).set_index('Party')
balances

,Initial Balance,Current Balance
Party,,
0,100000,100000
1,100000,100000
2,100000,100000
3,100000,100000
4,100000,100000
5,100000,100000
6,100000,100000
7,100000,100000
8,100000,100000


In [798]:
balances['Current Balance'] = balances['Current Balance'] - new_obligations

In [799]:
balances

,Initial Balance,Current Balance
Party,,
0,100000,99998.048605
1,100000,100012.936968
2,100000,100032.261396
3,100000,99991.691097
4,100000,100000.501654
5,100000,99999.672528
6,100000,99995.508473
7,100000,100004.397202
8,100000,99969.616163
